In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, month

In [3]:
spark = SparkSession.builder.appName("OptimizeI").getOrCreate()


In [4]:
%cd c:/sb/projects/spark-optimization

c:\sb\projects\spark-optimization


In [5]:
base_path = os.getcwd()

In [6]:
print (base_path)


c:\sb\projects\spark-optimization


In [7]:
answers_input_path = os.path.join(base_path, 'data\\answers')

In [8]:
print(answers_input_path)

c:\sb\projects\spark-optimization\data\answers


In [9]:
questions_input_path = os.path.join(base_path, 'data\\questions')

In [10]:
print(questions_input_path)

c:\sb\projects\spark-optimization\data\questions


In [11]:
answersDF = spark.read.option('path', answers_input_path).load()

In [12]:
answersDF.show(5)

+-----------+---------+--------------------+--------+-------+-----+
|question_id|answer_id|       creation_date|comments|user_id|score|
+-----------+---------+--------------------+--------+-------+-----+
|     226592|   226595|2015-12-29 13:46:...|       3|  82798|    2|
|     388057|   388062|2018-02-22 08:52:...|       8|    520|   21|
|     293286|   293305|2016-11-17 11:35:...|       0|  47472|    2|
|     442499|   442503|2018-11-21 20:34:...|       0| 137289|    0|
|     293009|   293031|2016-11-16 03:36:...|       0|  83721|    0|
+-----------+---------+--------------------+--------+-------+-----+
only showing top 5 rows



In [13]:
questionsDF = spark.read.option('path', questions_input_path).load()

In [14]:
questionsDF.show(5)

+-----------+--------------------+--------------------+--------------------+------------------+--------+-------+-----+
|question_id|                tags|       creation_date|               title|accepted_answer_id|comments|user_id|views|
+-----------+--------------------+--------------------+--------------------+------------------+--------+-------+-----+
|     382738|[optics, waves, f...|2018-01-27 21:22:...|What is the pseud...|            382772|       0|  76347|   32|
|     370717|[field-theory, de...|2017-11-24 23:09:...|What is the defin...|              null|       1|  75085|   82|
|     339944|[general-relativi...|2017-06-17 10:32:...|Could gravitation...|              null|      13| 116137|  333|
|     233852|[homework-and-exe...|2016-02-04 11:19:...|When does travell...|              null|       9|  95831|  185|
|     294165|[quantum-mechanic...|2016-11-22 01:39:...|Time-dependent qu...|              null|       1| 118807|   56|
+-----------+--------------------+--------------

In [15]:
answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month').agg(count('*').alias('cnt'))

In [16]:
answers_month.show(5)

+-----------+-----+---+
|question_id|month|cnt|
+-----------+-----+---+
|     358894|    9|  5|
|     332782|    5|  2|
|     281552|    9|  2|
|     332224|    5|  1|
|     395851|    3|  3|
+-----------+-----+---+
only showing top 5 rows



In [17]:
resultDF = questionsDF.join(answers_month, 'question_id').select('question_id', 'creation_date', 'title', 'month', 'cnt')

In [18]:
resultDF.orderBy('question_id', 'month').show()

+-----------+--------------------+--------------------+-----+---+
|question_id|       creation_date|               title|month|cnt|
+-----------+--------------------+--------------------+-----+---+
|     155989|2014-12-31 15:59:...|Frost bubble form...|    2|  1|
|     155989|2014-12-31 15:59:...|Frost bubble form...|   12|  1|
|     155990|2014-12-31 16:51:...|The abstract spac...|   12|  2|
|     155992|2014-12-31 17:44:...|centrifugal force...|   12|  1|
|     155993|2014-12-31 17:56:...|How can I estimat...|    1|  1|
|     155995|2014-12-31 19:16:...|Why should a solu...|    1|  2|
|     155995|2014-12-31 19:16:...|Why should a solu...|   12|  1|
|     155996|2014-12-31 20:06:...|Why do we assume ...|    1|  1|
|     155996|2014-12-31 20:06:...|Why do we assume ...|    2|  1|
|     155996|2014-12-31 20:06:...|Why do we assume ...|   11|  1|
|     155996|2014-12-31 20:06:...|Why do we assume ...|   12|  1|
|     155997|2014-12-31 20:26:...|Why do square sha...|    1|  2|
|     1559

In [19]:
'''
Task:

see the query plan of the previous result and rewrite the query to optimize it
'''

'\nTask:\n\nsee the query plan of the previous result and rewrite the query to optimize it\n'

In [20]:
resultDF.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [question_id#37L, creation_date#39, title#40, month#94, cnt#110L]
   +- BroadcastHashJoin [question_id#37L], [question_id#0L], Inner, BuildRight, false
      :- Filter isnotnull(question_id#37L)
      :  +- FileScan parquet [question_id#37L,creation_date#39,title#40] Batched: true, DataFilters: [isnotnull(question_id#37L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/c:/sb/projects/spark-optimization/data/questions], PartitionFilters: [], PushedFilters: [IsNotNull(question_id)], ReadSchema: struct<question_id:bigint,creation_date:timestamp,title:string>
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true]),false), [id=#273]
         +- HashAggregate(keys=[question_id#0L, month#94], functions=[count(1)])
            +- Exchange hashpartitioning(question_id#0L, month#94, 200), ENSURE_REQUIREMENTS, [id=#270]
               +- HashAggregate(keys=[question_id#0L, month#94], funct